In [10]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi,cos,sin
from scipy.optimize import least_squares

In [11]:
#将从示教器读取的数据转化为旋转矩阵和平移矩阵
def data_transform(M):
    t1,t2,t3,q0,q1,q2,q3,=M
    x11=q0**2+q1**2-q2**2-q3**2
    x12=2*(q1*q2-q0*q3)
    x13=2*(q1*q3+q0*q2)
    x14=t1
    x21=2*(q1*q2+q0*q3)
    x22=q0**2-q1**2+q2**2-q3**2
    x23=2*(q2*q3-q0*q1)
    x24=t2
    x31=2*(q1*q3-q0*q2)
    x32=2*(q2*q3+q0*q1)
    x33=q0**2-q1**2-q2**2+q3**2
    x34=t3
    TR=np.array([[x11,x12,x13,x14],[x21,x22,x23,x24],[x31,x32,x33,x34],[0,0,0,1]])
    return TR

In [12]:
def calculate_P0(TB,X,P):
    P0=np.dot(TB,np.dot(X,P))
    return P0
    

In [13]:
#主程序
def main():
    Pose1=[418.411,8.402,86.912,0.01528,0.99755,0.0599,0.03273]
    Pose2=[409.802,32.51,111.412,0.02784,0.99863,0.04425,0]
    Pose3=[401.502,25.411,108.519,0.02555,0.85545,0.51716,-0.01015]
    Pose4=[391.503,19.951,87.942,0.0119,-0.91629,-0.38719,0.10177]
    Pose5=[427.642,17.613,134.321,0.01256,-0.80777,-0.58894,0.02226]
    #机械臂末端坐标系相比于机械臂末端坐标系的变换矩阵
    TB1=data_transform(Pose1)
    TB2=data_transform(Pose2)
    TB3=data_transform(Pose3)
    TB4=data_transform(Pose4)
    TB5=data_transform(Pose5)
    R=246
    #标定板固定点相对于激光雷达的变化矩阵
    P1=np.array([13.00,5.31,92.69,1]).reshape(-1,1)
    P2=np.array([-12.01,4.52,67.12,1]).reshape(-1,1)
    P3=np.array([-11.50,5.21,70.25,1]).reshape(-1,1)
    P4=np.array([-1.701,-4.52,93.2,1]).reshape(-1,1)
    P5=np.array([19.7,5.23,43.5,1]).reshape(-1,1)
    A=[TB1,TB2,TB3,TB4,TB5]
    B=[P1,P2,P3,P4,P5]
    def func(x):
        q0,q1,q2,q3,t1,t2,t3=x
        X=data_transform(x)
        A1=np.dot(A[0],np.dot(X,B[0]))-np.dot(A[1],np.dot(X,B[1]))
        A2=np.dot(A[1],np.dot(X,B[1]))-np.dot(A[2],np.dot(X,B[2]))
        A3=np.dot(A[2],np.dot(X,B[2]))-np.dot(A[3],np.dot(X,B[3]))
        A4=np.dot(A[3],np.dot(X,B[3]))-np.dot(A[4],np.dot(X,B[4]))
        f=[A1[0,0],A1[1,0],A1[2,0],A2[0,0],A2[1,0],A2[2,0],A3[0,0],A3[1,0],A3[2,0],A4[0,0],A4[1,0],A4[2,0]]
        return f
    initial_guess=[0.5,0.5,0.5,0.5,0.01,0.01,0.01]
    result=least_squares(func,initial_guess)
    print("t1=",result.x[0])
    print("t2=",result.x[1])
    print("t3=",result.x[1])
    print("q0=",result.x[3])
    print("q1=",result.x[4])
    print("q2=",result.x[5])
    print("q3=",result.x[6])
    #计算出最后的标定矩阵
    X=data_transform(result.x)
    P01=calculate_P0(TB1,X,P1)
    P02=calculate_P0(TB2,X,P2)
    P03=calculate_P0(TB3,X,P3)
    P04=calculate_P0(TB4,X,P4)
    P05=calculate_P0(TB5,X,P5)
    error1=P01-P01
    error2=P02-P01
    error3=P03-P01
    error4=P04-P01
    error5=P05-P01
    print("error1=")
    print(error1)
    print("error2=")
    print(error2)
    print("error3=")
    print(error3)
    print("error4=")
    print(error4)
    print("error5=")
    print(error5)
main()

t1= 13.949546011973274
t2= -0.2506825543090668
t3= -0.2506825543090668
q0= -0.031070300376600055
q1= -0.710615200985581
q2= 0.6936192243772807
q3= 0.04993603858851841
error1=
[[0.]
 [0.]
 [0.]
 [0.]]
error2=
[[1.12864037]
 [0.89103725]
 [0.07776613]
 [0.        ]]
error3=
[[ 1.36082189]
 [ 1.46716448]
 [-0.13716365]
 [ 0.        ]]
error4=
[[ 0.82206274]
 [ 5.68236132]
 [-0.24081077]
 [ 0.        ]]
error5=
[[0.5524059 ]
 [6.80029492]
 [0.67142861]
 [0.        ]]
